!*********************************************************************!

!

!   This program calculates the unpolarized absorption spectrum and 

!   band dispersion of a linear, 1D aggregate using the Frenkel/

!   charge-transfer Holstein model. See the README file for more 

!   information.

!                                                   Written By:

!                                                   Nicholas Hestand

!*********************************************************************!

In [2]:
import sys
import re
from os import path
import numpy as np

# read in parameters
!*********************************************************************!

!   subroutine reads parameters from user input file 

!   most variables reside in the commonvar module and are used

!   throughout the program in various subroutines

!*********************************************************************!

In [ ]:
def read_in_para():
#subroutine read_in_para()

    #logical         exists
    #character*100   buff, label, fname
    #integer         fno, ios, line, pos, errstat
    #parameter       (fno = 201)
    
    #get the input file name as the first argument from command line
    try:
        fname=sys.argv[1]
    except ValueError:
    #if given otherwise use default parameters    
        if ValueError !=0:
            print('No control file given. Using default parameters')
            c1010
    #check that the given input file exists, abort if not
        if not path.exists(fname):
            print('Input file not found...aborting')
            return
    #open the user input file and read in the parameters
    with open(fname,'r') as f:
        #ios = 0  #the in/out status
        #line = 0 #the current line number
        print('Reading the input file...','\n','**********************************','\n','**********************************')
        r = re.compile("([a-zA-Z]+)([0-9|True|False]+)")
        parameters=['task_title','nmol','vibmax','hw','lambda_n','lambda_c','lambda_a','JCoul','ES1','te','th','ECT','ECTInf','one_state','two_state','ct_state','abs_lw','esnum']
        parameters={i:None for i in parameters}

        for buff in f.readlines():#!continue the loop until end of file,!read a line
            label,buff= r.match(line)#parse the line into a label and parameter
            if label.startswith('#'): #treat as a comment
                continue
            else:#find the label and assign the appropriate value to the variable
                try:
                    parameters[label]=buff
                except:
                    input('invalid label at line','\n',buff,'\n','press enter to continue or ctrl+c to abort')
    print('**********************************'+'\n'+'**********************************')
    return parameters
    
def c1010():
    print('Calculating derived parameters in units of hw.')

    #normalize parameters to units of hw    
    parameters['JCoul']  = parameters['JCoul'] /  parameters['hw']
    parameters['ES1']    = parameters['ES1'] /    parameters['hw']
    parameters['te']     = parameters['te'] /     parameters['hw']
    parameters['th']     = parameters['th'] /     parameters['hw']
    parameters['ECT']    = parameters['ECT'] /    parameters['hw']
    parameters['ECTInf'] = parameters['ECTInf'] / parameters['hw']
    parameters['abs_lw'] = parameters['abs_lw'] / parameters['hw']
    
    #Set all Huang-Rhys factors to zero if vibmax is zero
    #This assumes that the user just wants to calculate the
    #free exciton properties
    if ( parameters['vibmax'] == 0 ):
        parameters['lambda_n'] = 0
        parameters['lambda_c'] = 0
        parameters['lambda_a'] = 0
        print('(a)', '>> vibmax is zero. Setting all lambda to zero')

    #set the maximum left and right displacement from a given molecule given periodic boundary conditions
    parameters['nlbnd']  =  -parameters['nmol']/2+(1-1*np.mod(parameters['nmol'],2))
    parameters['nubnd'] =   parameters['nmol']/2
return parameters


In [ ]:
def exciton_main():
    #read the user input file and set simulation parameters
    read_in_para()

In [ ]:

#program exciton_main
    use commonvar
    implicit none

    integer k   !the k-index
    
    #!read the user input file and set simulation parameters
    call read_in_para()     
   
    !index the multiparticle basis set
    kount = 0
    if ( one_state ) call index_1p()
    if ( two_state ) call index_2p()
    if ( ct_state ) call index_ct()
    
    !make sure the number of requested eigenstates is less than the
    !total number of eigenstates possible
    esnum = min(esnum,kount) 

    !build the franck-condon table for the vibrational overlap factors
    call set_fctable()   
    
    !allocate space for the Hamiltonian matrix and eigenvalue array
    allocate ( h(kount,kount) ) !the Hamiltonian
    allocate ( eval(kount) )    !eigenvalues


    print'(a)', ' Will now build the Hamiltonian, the'//&
                   ' diminsion of each k-block is:'
    print'(i6)',  kount
    print*, '**********************************'//&
            '**********************************'

    !build each k-block of the Hamiltonian diagonalize,
    !and calculate the observables
    do k = nlbnd, nubnd

        !initialize hamiltonian to zero
        h = complex_zero
        
        !build the hamiltonian
        if ( one_state ) call build_h1p(k)
        if ( two_state ) call build_h2p(k)
        if ( one_state .and. two_state ) call build_h1p2p(k)
        if ( ct_state ) call build_hct(k)
        if ( one_state .and. ct_state ) call build_h1pct(k)
        if ( two_state .and. ct_state ) call build_h2pct(k)

        !diagonalize the hamiltonian
        if ( k == 0 .or. esnum == kount ) then
            call diagonalize(h, kount, eval, 'A', kount )
        else
            call diagonalize(h, kount, eval, 'I', esnum)
        end if   
        
        !calculate absorption spectrum
        !(only k=0 absorbes assuming parallel dipoles)
        if ( k == 0 ) then
            call absorption()
        end if
        call dispersion(k)
        
        print*, 'Done with wavevector k: ', k
    end do

    !write the parameter file
    call para_out()
    print*, '**********************************'//&
            '**********************************'
    print*, 'Program exited successfully.'

end program
